Empleando los datos del archivo './src/crypto_currency'.

- Cargue los datos en una base de datos apropiada.
- Lea mediante pandas las bases cargadas
- Emplee un metodo de concatenación para unificar la información.(debe considerar que se añadirá una columna adicional que especifique el tipo de moneda)
- Genere un agrupamiento de información a manera de obtener un resumen de los datos.
- Almacene dichos datos en un reporte excel.
- Apoyandose del ejercicio 2. Cree una imagen que sea guarda en el archivo excel
- Emplee un método de envio de correos



In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image

import pandas as pd
import xlsxwriter
import sqlite3
import os

In [9]:
DB = 'cripto_currency.db'
TABLE_NAME_1 = 'BTC'
TABLE_NAME_2 = 'DOGE'
TABLE_NAME_3 = 'USDT'

In [10]:
def almacenar_pandas_to_sql(df: pd.DataFrame, database_name:str, table_name:str)->None:
  """Procesamiento datos candidatos para almacenarlos sobre db """

  # rename columns
  column_rename = {c: c.replace(' ', '') for c in df.columns}
  df.rename(column_rename,axis='columns', inplace=True)

  # almaceno sobre db
  conn = sqlite3.connect(database_name)
  df.to_sql(table_name, conn, if_exists='replace', index=False)
  conn.close()

  #
  sql_table_schema = f'{database_name}.{table_name}'
  cantidad_registros = df.shape[0]
  print(f'Se almacenaron {cantidad_registros} sobre tabla sql {sql_table_schema}')

In [11]:
path = './src/cripto_currency.xlsx'
df1 = pd.read_excel(path, sheet_name="BTC-USD")
df2 = pd.read_excel(path, sheet_name="DOGE-USD")
df3 = pd.read_excel(path, sheet_name="USDT-USD")

In [12]:
almacenar_pandas_to_sql(df1, DB , TABLE_NAME_1)
almacenar_pandas_to_sql(df2, DB , TABLE_NAME_2)
almacenar_pandas_to_sql(df2, DB , TABLE_NAME_3)

Se almacenaron 366 sobre tabla sql cripto_currency.db.BTC
Se almacenaron 366 sobre tabla sql cripto_currency.db.DOGE
Se almacenaron 366 sobre tabla sql cripto_currency.db.USDT


In [13]:
with sqlite3.connect(DB) as conn:
  df1 = pd.read_sql_query(f'select * from {TABLE_NAME_1}', conn)

In [14]:
with sqlite3.connect(DB) as conn:
  df2 = pd.read_sql_query(f'select * from {TABLE_NAME_2}', conn)

In [15]:
with sqlite3.connect(DB) as conn:
  df3 = pd.read_sql_query(f'select * from {TABLE_NAME_3}', conn)

In [26]:
df1

,Date,Open,High,Low,Close,AdjClose,Volume,Moneda
0,2022-10-19 00:00:00,19335.027344,19348.416016,19127.687500,19139.535156,19139.535156,22425387184,btc-usd
1,2022-10-20 00:00:00,19138.085938,19315.199219,18971.458984,19053.740234,19053.740234,24493974420,btc-usd
2,2022-10-21 00:00:00,19053.203125,19237.384766,18770.970703,19172.468750,19172.468750,32459287866,btc-usd
3,2022-10-22 00:00:00,19172.380859,19248.068359,19132.244141,19208.189453,19208.189453,16104440957,btc-usd
4,2022-10-23 00:00:00,19207.734375,19646.652344,19124.197266,19567.007813,19567.007813,22128794335,btc-usd
...,...,...,...,...,...,...,...,...
361,2023-10-15 00:00:00,26858.011719,27289.169922,26817.894531,27159.652344,27159.652344,7098201980,btc-usd
362,2023-10-16 00:00:00,27162.628906,29448.138672,27130.472656,28519.466797,28519.466797,27833876539,btc-usd
363,2023-10-17 00:00:00,28522.097656,28618.751953,28110.185547,28415.748047,28415.748047,14872527508,btc-usd
364,2023-10-18 00:00:00,28413.531250,28889.009766,28174.251953,28328.341797,28328.341797,12724128586,btc-usd


In [37]:
with sqlite3.connect(DB) as conn:
    df1['Moneda'] = 'btc-usd'
    df2['Moneda'] = 'doge-usd'
    df3['Moneda'] = 'usdt-usd'

    # Concatena los DataFrames
    df = pd.concat([df1, df2, df3])

In [38]:
with sqlite3.connect(DB) as conn:
    df.to_sql('TABLA_UNIFICADA', conn, if_exists='replace', index=False)

In [39]:
with sqlite3.connect(DB) as conn:
    cur = conn.cursor()
    # Write your SQL query
    query = '''SELECT * FROM TABLA_UNIFICADA
    WHERE Moneda == "btc-usd" '''

    filterDf = pd.read_sql_query(query, conn)
   

In [42]:
filterDf['Date'] = pd.to_datetime(filterDf['Date'])

In [47]:
summary = filterDf.describe()
summary.to_excel('./reportes/reporteBTC.xlsx')